# Lab Assignment 6
#### This Lab task is to completed individually, Please Add your Name and ID Below, please comment your code thoroughly and name your variables wisely. (Grade out of 17, 3 marks for commented organized code.)
- Name: Francisco Javier Diaz Roa
- ID: 8975938

### Question 1-   (5)
Check the files inside microbiome folder, (MID1, MID2, ... , MID9.xls files), you need to combine those files into single dataframe.<br>
Then You will need to add to this dataframe the relevant metadata information from metadata.csv <br>
Each folder represents information from different bacteria, MID1, MID2, ...
<br>
__Inspect the files before answering the question__


In [2]:
!pip install xlrd


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip install --upgrade pip


In [3]:
# combine the MID1, MID2, ... , MID9.xls files into a single dataframe


import pandas as pd
import os

# get the list of files in the current directory
files = os.listdir()


# Initialize an empty list to store DataFrames
data_frames = []

# filter out the files that are not .xls files
files = [f for f in files if f.endswith('.xls')]

# read the metadata file
metadata = pd.read_excel('metadata.xls')

# read each file and append the data to the dataframe
for f in files:
    # ignore the metadata.xls file
    if f == 'metadata.xls' :
        continue
    data = pd.read_excel(f)
    # name the two columns
    data.columns = ['name', 'count']

    # get the last digit of the file name
    n = f[-5]

    barcode = metadata['BARCODE'][int(n) - 1]
    group = metadata['GROUP'][int(n) - 1]
    sample = metadata['SAMPLE'][int(n) - 1]

    # Add metadata information to the data
    data['barcode'] = barcode
    data['group'] = group
    data['sample'] = sample

    # append the data to the list
    data_frames.append(data)

# concatenate the dataframes
df = pd.concat(data_frames)
print(df.head())

# save the dataframe to a csv file
df.to_csv('combined_data.csv', index=False)








                                                name  count barcode  group  \
0  Archaea "Crenarchaeota" Thermoprotei Desulfuro...      1    MID6  NEC 1   
1  Archaea "Crenarchaeota" Thermoprotei Desulfuro...      1    MID6  NEC 1   
2  Archaea "Crenarchaeota" Thermoprotei Desulfuro...      2    MID6  NEC 1   
3  Archaea "Crenarchaeota" Thermoprotei Sulfoloba...      5    MID6  NEC 1   
4  Archaea "Crenarchaeota" Thermoprotei Sulfoloba...      4    MID6  NEC 1   

  sample  
0  stool  
1  stool  
2  stool  
3  stool  
4  stool  


### Question 2: Approximate String Matching
2.1- Implement FROM SCRATCH the approximate string matching functionality using Levenshtein Distance (6 marks)
- Levenshtein Distance between two strings a,b of length |a| and |b| is calculated by creating a matrix of dimensions |a+1|x|b+1| and then filling the matrix by the following equation:<br>
lev<sub>a,b</sub>(i,j) = <br>
a. max(i,j) IF min(i,j)=0<br>
b. minimum of:    
    <ol> <li>lev<sub>a,b</sub>(i-1,j)+1</li>
    <li>lev<sub>a,b</sub>(i,j-1)+1</li>
    <li>lev<sub>a,b</sub>(i-1,j-1)+ 
            *1 if (a<sub>i</sub> != b<sub>j</sub>), 0 otherwise*</li> </ol>


In [4]:
def calculate_levenshtein_distance(string_a, string_b):
    ## TODO: Write the functionality that calculates how many edit operations are required to change one string into another
    ## Example: Distance "HONDA" to "HYUNDAI" is 3, where the operations are adding "Y" after H in Honda
    ## Then replacing O with Y, then keeping N, keeping D, keeping A, adding I.
    # crete a function to calculate_levenshtein_distance wich takes two strings as input
    
    # Create the matrix of the size of the two strings
    matrix = [[0 for i in range(len(string_b)+1)] for j in range(len(string_a)+1)]

    # Fill the matrix with the minimum of the three possible values
    for i in range(1, len(string_a)+1):
        matrix[i][0] = i
        for j in range(0, len(string_b)+1):
            if j==0:
                matrix[0][j] = j
                continue
            insertion = matrix[i][j-1] + 1
            deletion = matrix[i-1][j] + 1
            substitution = matrix[i-1][j-1] + (0 if string_a[i-1] == string_b[j-1] else 1)
            matrix[i][j] = min(insertion, deletion, substitution)

    print(matrix)

    # Return the value of the last element of the matrix
    return matrix[-1][-1]

# Test the function with the example
print(calculate_levenshtein_distance("HONDA", "HYUNDAI")) # Expected 3

[[0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 1, 1, 1, 1, 1, 1], [2, 1, 1, 2, 2, 2, 2, 2], [3, 2, 2, 2, 2, 3, 3, 3], [4, 3, 3, 3, 3, 2, 3, 4], [5, 4, 4, 4, 4, 3, 2, 3]]
3


2.2- Using the implemented Levenshtein's distance, merge the following dataframes based on the closest match for the "Name" attribute. (You will need to find first the matching pairs) (3 marks)

In [5]:
# First Dataframe
data1 = {'ID1': [1, 2, 3, 4],
         'Name1': ['John Smith', 'Alice Johnson', 'Bob Marley', 'David Doe']}
df1 = pd.DataFrame(data1)
# Second Dataframe
data2 = {'ID2': [101, 102, 103, 104,105],
         'Name2': ['Jon Smith', 'Alyce Jonson', 'Robert Marley', 'Dave D.', 'Rob Marly']}
df2 = pd.DataFrame(data2)

# Initialize an empty list to store the closest matches
matches = []

# Calculate the Levenshtein distance for all pairs
for index1, row1 in df1.iterrows():
    # Initialize the minimum distance to infinity and the best match to None
    # Infinity is used to make sure that the first distance will be less than the minimum distance
    # why not 0? because the distance can be 0 if the two strings are the same which is not the case here
    min_distance = float('inf')
    # best_match = None is used to make sure that the first match will be stored
    best_match = None
    # Iterate over the second DataFrame
    for index2, row2 in df2.iterrows():
        distance = calculate_levenshtein_distance(row1['Name1'], row2['Name2'])
        # If the distance is less than the minimum distance, update the minimum distance and the best match
        if distance < min_distance:
            min_distance = distance
            best_match = row2
    matches.append({
        'ID1': row1['ID1'],
        'Name1': row1['Name1'],
        'ID2': best_match['ID2'],
        'Name2': best_match['Name2'],
        'Levenshtein_Distance': min_distance
    })

# Create a DataFrame from the matches
df_matches = pd.DataFrame(matches)  
print(df_matches)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 1, 1, 1, 1, 1, 1, 1, 1], [2, 1, 0, 1, 2, 2, 2, 2, 2, 2], [3, 2, 1, 1, 2, 3, 3, 3, 3, 2], [4, 3, 2, 1, 2, 3, 4, 4, 4, 3], [5, 4, 3, 2, 1, 2, 3, 4, 5, 4], [6, 5, 4, 3, 2, 1, 2, 3, 4, 5], [7, 6, 5, 4, 3, 2, 1, 2, 3, 4], [8, 7, 6, 5, 4, 3, 2, 1, 2, 3], [9, 8, 7, 6, 5, 4, 3, 2, 1, 2], [10, 9, 8, 7, 6, 5, 4, 3, 2, 1]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1], [2, 2, 2, 2, 2, 2, 2, 1, 0, 1, 2, 1, 2], [3, 3, 3, 3, 3, 3, 3, 2, 1, 1, 2, 2, 2], [4, 4, 4, 4, 4, 4, 4, 3, 2, 1, 2, 3, 2], [5, 5, 5, 5, 5, 5, 4, 4, 3, 2, 2, 3, 3], [6, 6, 6, 6, 6, 6, 5, 5, 4, 3, 3, 3, 4], [7, 7, 7, 7, 7, 7, 6, 6, 5, 4, 4, 4, 4], [8, 8, 8, 8, 8, 8, 7, 7, 6, 5, 5, 5, 5], [9, 9, 9, 9, 9, 9, 8, 8, 7, 6, 6, 6, 6], [10, 10, 10, 10, 10, 10, 9, 9, 8, 7, 7, 7, 7]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [3, 3, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], [4, 4, 3, 3